In [ ]:
# montar a drive e imports
import pandas as pd
import time
import random

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# source http://snap.stanford.edu/data/ego-Facebook.html
# File path
filepath = '/content/drive/MyDrive/Colab Notebooks/Computação na Cloud (CC)/gplus_combined.csv'
# pandas to read dataset
dataset = pd.read_csv(filepath, delimiter=' ')
print("Dataset\n", dataset)

x = dataset.iloc[:, 0]
y = dataset.iloc[:, 1]

Dataset
           116374117927631468606  101765416973555767821
0         112188647432305746617  107727150903234299458
1         116719211656774388392  100432456209427807893
2         117421021456205115327  101096322838605097368
3         116407635616074189669  113556266482860931616
4         105706178492556563330  111169963967137030210
...                         ...                    ...
30494860  118379821279745746467  102953973878117962383
30494861  118379821279745746467  116863879615717543881
30494862  118379821279745746467  102977664233671324384
30494863  118379821279745746467  106299956776996674942
30494864  118379821279745746467  109995768273040439562

[30494865 rows x 2 columns]


In [ ]:
# Ids to search friends
# Change this value to get the number of IDs to find the mutual friends
n_ids = 3
friend = []
count = []

for i in range(n_ids):
  friend.append(x[random.randint(0, 30494865)])
  count.append(0)

'''
# Exemples for debugging:
# 2 IDs
109123727761757726835
100722679426961808186
_____________________
2356 mutual friends

# 3 IDs
103652441454188509895
116635061583722144944
113882113745075873153
______________________
1405 mutual friends

'''

t0 = time.time()

# Counting nº of friends to each ID
# Each line is: friend0 friend_0
#               friend0 friend_1
#               friend0 friend_2

# Computational Complexity: O(n^m)
for i in x:
  for j in range(len(friend)):
    if friend[j] == i:
      count[j] += 1
t1 = time.time()

print('Delta t(s) = ', (t1-t0))
for i in range(len(friend)):
  print('Nº of friends of %s: %d' %(friend[i], count[i]))


Delta t(s) =  26.674137830734253
Nº of friends of 103652441454188509895: 7680
Nº of friends of 116635061583722144944: 5841
Nº of friends of 113882113745075873153: 4577


In [ ]:
# Now counts the number of mutual friends
# Computational Complexity: O(n + n*m)
# Mutual Friends will be
mutual_friends = []
friends_in_common = 0
t0 = time.time()

# Fetch the friends of each and appends them to a list
for i in range(len(x)):
  for j in friend:
    if j == x[i]:
      s = y[i] + '\t1'
      mutual_friends.append(s)
  
t1 = time.time()

# Now we sort the list (matches the MapReduce Job on Hadoop)
mutual_friends.sort()

# verify if there is any mutual friend
line = mutual_friends[0]
current_id, current_count = line.split('\t', 1)
current_count = int(current_count)
for i in range(1, len(mutual_friends)):
  line = mutual_friends[i]
  id, count = line.split('\t', 1)
  count = int(count)
  if current_id == id:
    current_count += 1
    if current_count == n_ids:
      friends_in_common += 1
  else:
    current_count = count
    current_id = id
  

t2 = time.time()

print('Delta fetch friends(s) = %.2f' %(t1-t0))
print('Delta mutual friends(s) = %.2f' %(t2-t1))
print('Delta Total(s) = %.2f' %(t2-t0))
print('Nº of mutual friends: ', friends_in_common)

Delta fetch friends(s) = 359.90
Delta mutual friends(s) = 0.03
Delta Total(s) = 359.92
Nº of mutual friends:  1405
